In [22]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.insert(0, '../')
from scripts.tracking import MatchTracking 
from scripts.pass_events import PassEvents
from scripts.ball_visualization import BallVisualization

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
data_directory = r'..\data\tracking'
file = 'g2312201_SecondSpectrum_tracking-produced.jsonl'
match_tracking_path = os.path.join(data_directory, file)
match_tracking = MatchTracking(tracking_file_path=match_tracking_path)

In [24]:
data_directory = r'..\data\statsbomb'
file = 'ManCity_AstonVilla_events.json'
events_path = os.path.join(data_directory, file)
pass_events = PassEvents(tracking_file_path=events_path)

In [25]:
pass_events.update_position(match_tracking)

In [26]:
pass_events.update_dataset_with_position(match_tracking)

In [27]:
pass_events.clean_dataset()

c:\Users\MatteoBENAUD\OneDrive - Ekimetrics\Bureau\hackathon City\ManHackathonIrsem\examples\..\scripts\pass_events.py:217: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modelling_df_home.loc[:,'completed'] = 0
c:\Users\MatteoBENAUD\OneDrive - Ekimetrics\Bureau\hackathon City\ManHackathonIrsem\examples\..\scripts\pass_events.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modelling_df_away.loc[:,'completed'] = 0


In [28]:
pass_events.df_pass_home.columns

Index(['period', 'gameClock', 'team.name', 'duration', 'x', 'y',
       'end_location_x', 'end_location_y', 'player.id', 'player.name',
       'pass.recipient.id', 'pass.recipient.name', 'player.jersey_nb',
       'pass.recipient.jersey_nb', 'pass.body_part.name', 'pass.height.name',
       'pass.outcome.name', 'jersey_number', 'x_passer', 'y_passer',
       'jersey_number_recipient', 'x_recipient', 'y_recipient',
       'coord_all_team', 'coord_all_adversary', 'completed'],
      dtype='object')

In [29]:
pass_events.get_pitch_dimensions(match_tracking)
pass_events.set_df_for_model()

In [37]:
pass_events.df_model.columns

Index(['period', 'gameClock', 'team.name', 'x_passer', 'y_passer',
       'x_recipient', 'y_recipient', 'player.jersey_nb',
       'pass.recipient.jersey_nb', 'coord_all_team', 'coord_all_adversary',
       'completed', 'distance_sideline', 'distance_goal', 'distance_opponent',
       'speed_passer', 'opponents_closer_to_goal',
       'distance_receiver_sideline', 'distance_receiver_goal',
       'distance_receiver_opponent', 'opponents_closer_to_goal_receiver',
       'speed_receiver', 'bypassed_opponents', 'angle', 'opponents_in_path',
       'nearest_defender_pass_line', 'distance_pass'],
      dtype='object')

In [40]:
pass_events.df_model = pass_events.df_model[pass_events.df_model['team.name']=='Manchester City WFC']

In [43]:
import json
data_directory = r'..\data\statsbomb'
file = 'ManCity_AstonVilla_lineups.json'
f = open(os.path.join(data_directory, file))
data = json.load(f)
mapping_jersey = pd.json_normalize(data[0], "lineup")
dict_mapping_jersey = dict(zip(mapping_jersey['jersey_number'],mapping_jersey['player_id']))
pass_events.df_model['player.id'] = pass_events.df_model['player.jersey_nb'].map(dict_mapping_jersey)
pass_events.df_model['pass.recipient.id'] = pass_events.df_model['pass.recipient.jersey_nb'].map(dict_mapping_jersey)

C:\Users\MatteoBENAUD\AppData\Local\Temp\ipykernel_15064\1935881879.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pass_events.df_model['player.id'] = pass_events.df_model['player.jersey_nb'].map(dict_mapping_jersey)
C:\Users\MatteoBENAUD\AppData\Local\Temp\ipykernel_15064\1935881879.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pass_events.df_model['pass.recipient.id'] = pass_events.df_model['pass.recipient.jersey_nb'].map(dict_mapping_jersey)


In [45]:
pass_events.df_model.to_excel('astonvilla_comparison.xlsx')

In [44]:
pass_events.df_model

,period,gameClock,team.name,x_passer,y_passer,x_recipient,y_recipient,player.jersey_nb,pass.recipient.jersey_nb,coord_all_team,...,distance_receiver_opponent,opponents_closer_to_goal_receiver,speed_receiver,bypassed_opponents,angle,opponents_in_path,nearest_defender_pass_line,distance_pass,player.id,pass.recipient.id
458,1,2850.187,Manchester City WFC,6.92,-30.75,-0.31,-16.73,11,21,"{14: [25.74, 3.21, 4.52], 25: [28.19, -27.33, ...",...,7.227704,7,1.20,0,117.279790,3,0.216172,15.774451,15555,25554
178,1,1012.530,Manchester City WFC,-2.96,5.12,-14.87,-25.61,25,11,"{14: [1.17, 26.36, 2.7], 25: [-2.96, 5.12, 2.8...",...,11.783463,9,2.12,0,-111.184794,2,2.922725,32.957260,25632,15555
370,1,2343.999,Manchester City WFC,5.82,-26.05,13.32,-25.67,21,11,"{14: [20.34, -5.45, 1.7000000000000002], 25: [...",...,2.510080,6,4.40,2,2.900506,1,0.016699,7.509620,25554,15555
369,1,2342.914,Manchester City WFC,13.82,-28.52,5.74,-25.19,11,21,"{14: [21.07, -5.25, 0.7000000000000001], 25: [...",...,0.408167,8,0.70,0,157.602035,1,0.327521,8.739296,15555,25554
187,1,1047.478,Manchester City WFC,-8.15,15.82,-19.96,11.83,7,21,"{14: [-13.6, 16.3, 3.99], 25: [3.83, 21.21, 1....",...,3.203951,7,3.80,0,-161.332497,2,0.135170,12.465801,19416,25554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2,1126.134,Manchester City WFC,2.97,-4.57,5.90,22.25,5,15,"{25: [-3.48, 9.69, 2.73], 9: [-21.87, 25.15, 1...",...,17.965603,1,3.41,1,83.765343,1,0.248747,26.979572,10252,10163
194,2,1125.194,Manchester City WFC,-6.76,-4.09,-4.26,0.53,11,5,"{25: [-4.21, 12.2, 2.8200000000000003], 9: [-2...",...,2.453120,1,4.57,1,61.581048,2,0.238567,5.253037,15555,10252
193,2,1123.610,Manchester City WFC,-8.95,3.94,8.29,4.90,7,11,"{25: [-3.16, 16.46, 3.02], 9: [-17.15, 27.51, ...",...,5.202125,0,0.68,4,3.187193,2,0.267868,17.266708,19416,15555
219,2,1192.371,Manchester City WFC,18.75,-20.94,6.33,8.58,5,21,"{25: [14.53, -4.11, 2.88], 9: [-6.65, 26.1, 1....",...,1.612514,3,3.68,0,112.818034,3,0.306367,32.026345,10252,25554
